<a href="https://colab.research.google.com/github/AnnaZhuravleva/avtobreja/blob/master/crop_at_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Here I merge results of differnet methods

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Extract graph based scores

In [0]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [3]:
!wget https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/seeds/service_sentiment_reclassified.csv

--2019-12-23 18:34:05--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/seeds/service_sentiment_reclassified.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23179 (23K) [text/plain]
Saving to: ‘service_sentiment_reclassified.csv’

service_sentiment_r 100%[===================>]  22.64K  --.-KB/s    in 0.01s   

2019-12-23 18:34:05 (1.84 MB/s) - ‘service_sentiment_reclassified.csv’ saved [23179/23179]



In [4]:
!wget https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/seeds/food_sentiment_reclassified.csv

--2019-12-23 18:34:08--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/seeds/food_sentiment_reclassified.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23179 (23K) [text/plain]
Saving to: ‘food_sentiment_reclassified.csv’

food_sentiment_recl 100%[===================>]  22.64K  --.-KB/s    in 0.01s   

2019-12-23 18:34:08 (1.90 MB/s) - ‘food_sentiment_reclassified.csv’ saved [23179/23179]



In [5]:
service_graph_scores_df = pd.read_csv("service_sentiment_reclassified.csv", sep='\t', names=['word', 'POS', 'score'])
service_graph_scores_df['lemma_POS'] = service_graph_scores_df['word'].apply(lambda s: s.strip('-').strip('.').lower()) + '_' + service_graph_scores_df['POS']
service_graph_scores_df = service_graph_scores_df.drop(['POS', 'word'], axis=1)
service_graph_scores_df.tail()

,score,lemma_POS
918,1,уютный_ADJ
919,1,свежий_ADJ
920,1,красивый_ADJ
921,1,сытный_ADJ
922,1,вкусный_ADJ


In [0]:
service_graph_scores = list(set([tuple([r[1], r[0]]) for r in service_graph_scores_df.values]))

In [7]:
food_graph_scores_df = pd.read_csv("food_sentiment_reclassified.csv", sep='\t', names=['word', 'POS', 'score'])
food_graph_scores_df['lemma_POS'] = food_graph_scores_df['word'].apply(lambda s: s.strip('-').strip('.').lower()) + '_' + food_graph_scores_df['POS']
food_graph_scores_df = food_graph_scores_df.drop(['POS', 'word'], axis=1)
food_graph_scores_df.tail()

,score,lemma_POS
918,1,уютный_ADJ
919,1,свежий_ADJ
920,1,красивый_ADJ
921,1,сытный_ADJ
922,1,вкусный_ADJ


In [0]:
food_graph_scores = list(set([tuple([r[1], r[0]]) for r in food_graph_scores_df.values]))

## Extract n-gram ranks

In [9]:
!wget https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/synt_3grams/service_trigrams.csv

--2019-12-23 18:34:12--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/synt_3grams/service_trigrams.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 461122 (450K) [text/plain]
Saving to: ‘service_trigrams.csv’

service_trigrams.cs 100%[===================>] 450.31K  --.-KB/s    in 0.06s   

2019-12-23 18:34:12 (7.59 MB/s) - ‘service_trigrams.csv’ saved [461122/461122]



In [10]:
!wget https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/synt_3grams/food_trigrams.csv

--2019-12-23 18:34:14--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/synt_3grams/food_trigrams.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462920 (452K) [text/plain]
Saving to: ‘food_trigrams.csv’

food_trigrams.csv   100%[===================>] 452.07K  --.-KB/s    in 0.07s   

2019-12-23 18:34:14 (6.04 MB/s) - ‘food_trigrams.csv’ saved [462920/462920]



In [11]:
service_n_grams = pd.read_csv('service_trigrams.csv', sep='\t', names=['lemma_POS', 'rank'])
service_n_grams.tail()

,lemma_POS,rank
8362,место_NOUN проведение_NOUN для_ADP,-76.696942
8363,место_NOUN город_NOUN,-78.811848
8364,место_NOUN обслуживание_NOUN с_ADP,-81.014264
8365,место_NOUN атмосфера_NOUN,-84.436807
8366,место_NOUN район_NOUN для_ADP,-96.897851


In [12]:
food_n_grams =  pd.read_csv('food_trigrams.csv', sep='\t', names=['lemma_POS', 'rank'])
food_n_grams.tail()

,lemma_POS,rank
8362,плохо_ADJ не_PART но_CCONJ,-114.743118
8363,плохо_ADJ очень_ADV,-114.901504
8364,плохо_ADJ не_PART,-117.746496
8365,есть_VERB плохо_ADV,-119.570822
8366,плохой_ADJ не_PART интерьер_NOUN,-123.060394


## Exstract combined CNN and vector ranks

In [21]:
!wget https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/word_lists/total_CNN%2BSAM/full_service_wo_mark.csv

--2019-12-23 18:39:42--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/word_lists/total_CNN%2BSAM/full_service_wo_mark.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1246393 (1.2M) [text/plain]
Saving to: ‘full_service_wo_mark.csv’

full_service_wo_mar 100%[===================>]   1.19M  --.-KB/s    in 0.07s   

2019-12-23 18:39:42 (16.2 MB/s) - ‘full_service_wo_mark.csv’ saved [1246393/1246393]



In [16]:
!wget https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/word_lists/total_CNN%2BSAM/full_food_wo_mark.csv

--2019-12-23 18:38:01--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/word_lists/total_CNN%2BSAM/full_food_wo_mark.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1301929 (1.2M) [text/plain]
Saving to: ‘full_food_wo_mark.csv.1’

full_food_wo_mark.c 100%[===================>]   1.24M  --.-KB/s    in 0.07s   

2019-12-23 18:38:01 (16.9 MB/s) - ‘full_food_wo_mark.csv.1’ saved [1301929/1301929]



In [23]:
service_combined =  pd.read_csv('full_service_wo_mark.csv', sep='\t', names=['lemma_POS', 'rank'])
service_combined.tail()

,lemma_POS,rank
29968,ласковый_ADJ,-110.269782
29969,любезный_ADJ,-111.020476
29970,дружелюбный_ADJ,-113.138820
29971,радушный_ADJ,-119.292144
29972,приветливый_ADJ,-120.355912


In [24]:
food_combined =  pd.read_csv('full_food_wo_mark.csv', sep='\t', names=['lemma_POS', 'rank'])
food_combined.tail()

,lemma_POS,rank
31260,необыкновенный_ADJ,-73.041091
31261,великолепный_ADJ,-76.084335
31262,живописный_ADJ,-76.777247
31263,обширный_ADJ,-77.776156
31264,величественный_ADJ,-80.639570


## Crop at 1%

### n-grams

#### service

In [25]:
service_n_grams.sort_values(['rank'], ascending=False)

,lemma_POS,rank
0,приветливый_ADJ доброжелательный_ADJ,216.582344
1,приветливый_ADJ внимательный_ADJ,210.865763
2,приветливый_ADJ вежливый_ADJ,199.615683
3,вежливый_ADJ приветливый_ADJ,199.615683
4,милый_ADJ приятный_ADJ,181.542751
...,...,...
8362,место_NOUN проведение_NOUN для_ADP,-76.696942
8363,место_NOUN город_NOUN,-78.811848
8364,место_NOUN обслуживание_NOUN с_ADP,-81.014264
8365,место_NOUN атмосфера_NOUN,-84.436807


In [26]:
len(service_n_grams) * 0.01

83.67

In [0]:
service_n_grams_sorted = list(service_n_grams.sort_values(['rank'], ascending=False)["lemma_POS"])

In [28]:
service_n_grams_good = service_n_grams_sorted[:int(len(service_n_grams) * 0.01)]
service_n_grams_bad = service_n_grams_sorted[-int(len(service_n_grams) * 0.01):]
service_n_grams_good[0]

'приветливый_ADJ доброжелательный_ADJ'

In [0]:
service_n_gram_scores = [(word, 1)  for word in service_n_grams_good] + [(word, 0)  for word in service_n_grams_bad]

In [0]:
def crop_and_score(df, ratio=0.01):
  df_sorted = list(df.sort_values(['rank'], ascending=False)["lemma_POS"])
  df_good = df_sorted[:int(len(df) * ratio)]
  df_bad = df_sorted[-int(len(df) * ratio):]
  return [(word, 1)  for word in df_good] + [(word, 0)  for word in df_bad]

In [31]:
service_n_gram_scores == crop_and_score(service_n_grams)

True

In [32]:
service_n_gram_scores_df = pd.DataFrame(service_n_gram_scores, columns =['lemma_POS', 'score']) 
service_n_gram_scores_df.tail()

,lemma_POS,score
161,место_NOUN проведение_NOUN для_ADP,0
162,место_NOUN город_NOUN,0
163,место_NOUN обслуживание_NOUN с_ADP,0
164,место_NOUN атмосфера_NOUN,0
165,место_NOUN район_NOUN для_ADP,0


#### food

In [33]:
food_n_gram_scores = crop_and_score(food_n_grams)
food_n_gram_scores_df = pd.DataFrame(food_n_gram_scores, columns =['lemma_POS', 'score']) 
food_n_gram_scores_df.tail()

,lemma_POS,score
161,плохо_ADJ не_PART но_CCONJ,0
162,плохо_ADJ очень_ADV,0
163,плохо_ADJ не_PART,0
164,есть_VERB плохо_ADV,0
165,плохой_ADJ не_PART интерьер_NOUN,0


### CNN + vector

In [34]:
service_combined_scores = crop_and_score(service_combined)
service_combined_scores_df = pd.DataFrame(service_combined_scores, columns =['lemma_POS', 'score']) 
service_combined_scores_df.tail()

,lemma_POS,score
593,ласковый_ADJ,0
594,любезный_ADJ,0
595,дружелюбный_ADJ,0
596,радушный_ADJ,0
597,приветливый_ADJ,0


In [35]:
food_combined_scores = crop_and_score(food_combined)
food_combined_scores_df = pd.DataFrame(food_combined_scores, columns =['lemma_POS', 'score']) 
food_combined_scores_df.tail()

,lemma_POS,score
619,необыкновенный_ADJ,0
620,великолепный_ADJ,0
621,живописный_ADJ,0
622,обширный_ADJ,0
623,величественный_ADJ,0


## Compare graph & combined

**CONTRADICTIONS WILL BE RESOLVED BY HAND**

In [0]:
def extend_vocabulary(scores, vocabulary):
  scores_dict = dict()
  for word, score in scores:
    scores_dict[word] = score
  for word in vocabulary:
    if word not in scores_dict:
      scores_dict[word] = np.nan
  return [el[1] for el in sorted(list(scores_dict.items()), key = lambda el: el[0])]

### Service

In [0]:
vocab_service = sorted(list(set([el[0] for el in service_combined_scores + service_graph_scores])))

In [38]:
service_df = pd.DataFrame(columns=['graph', 'combined'], index=vocab_service)
service_df['graph'] = extend_vocabulary(service_graph_scores, vocab_service)
service_df['combined'] = extend_vocabulary(service_combined_scores, vocab_service)
service_df.tail()

,graph,combined
японский_ADJ,0.0,NaN
японско-минималистический_ADJ,0.0,NaN
япошка_NOUN,NaN,1.0
яркий_ADJ,1.0,NaN
ясный_ADJ,0.0,NaN


In [39]:
service_df['ok'] = service_df['graph'] == service_df['combined']
service_df[service_df['ok'] == False].dropna()

,graph,combined,ok
ароматный_ADJ,1.0,0.0,False
вежливый_ADJ,1.0,0.0,False
веселый_ADJ,1.0,0.0,False
внимательный_ADJ,1.0,0.0,False
доброжелательный_ADJ,1.0,0.0,False
дорогой_ADJ,1.0,0.0,False
дружелюбный_ADJ,1.0,0.0,False
интересный_ADJ,1.0,0.0,False
комфортный_ADJ,1.0,0.0,False
красивый_ADJ,1.0,0.0,False


In [40]:
resolve_service = list(service_df[service_df['ok'] == False].dropna().index)
resolve_service

['ароматный_ADJ',
 'вежливый_ADJ',
 'веселый_ADJ',
 'внимательный_ADJ',
 'доброжелательный_ADJ',
 'дорогой_ADJ',
 'дружелюбный_ADJ',
 'интересный_ADJ',
 'комфортный_ADJ',
 'красивый_ADJ',
 'легкий_ADJ',
 'любезный_ADJ',
 'мягкий_ADJ',
 'недорогой_ADJ',
 'ненавязчивый_ADJ',
 'обходительный_ADJ',
 'оригинальный_ADJ',
 'отзывчивый_ADJ',
 'предупредительный_ADJ',
 'приветливый_ADJ',
 'светлый_ADJ',
 'симпатичный_ADJ',
 'спокойный_ADJ',
 'теплый_ADJ',
 'тихий_ADJ',
 'удивительный_ADJ',
 'удобный_ADJ',
 'уютно_ADJ',
 'щедрый_ADJ']

In [41]:
ok_service_df = service_df.drop(index=resolve_service).fillna(0)
ok_service_df = pd.DataFrame(ok_service_df['graph'] + ok_service_df['combined'], columns=['correct'])
ok_service_df.tail()

,correct
японский_ADJ,0.0
японско-минималистический_ADJ,0.0
япошка_NOUN,1.0
яркий_ADJ,1.0
ясный_ADJ,0.0


In [42]:
resolved_service_df = pd.DataFrame([1.0]*len(resolve_service), columns=['correct'], index=resolve_service)
resolved_service_df.tail()

,correct
тихий_ADJ,1.0
удивительный_ADJ,1.0
удобный_ADJ,1.0
уютно_ADJ,1.0
щедрый_ADJ,1.0


In [43]:
ok_service_df.append(resolved_service_df)

,correct
авторский_ADJ,0.0
адекватный_ADJ,0.0
адоптировать_ADJ,0.0
азиатский_ADJ,0.0
аккуратненький_ADJ,0.0
...,...
тихий_ADJ,1.0
удивительный_ADJ,1.0
удобный_ADJ,1.0
уютно_ADJ,1.0


In [0]:
ok_service_scores = [tuple(r) for r in ok_service_df.append(resolved_service_df).reset_index().values]

### Food

In [0]:
vocab_food = sorted(list(set([el[0] for el in food_combined_scores + food_graph_scores])))

In [46]:
food_df = pd.DataFrame(columns=['graph', 'combined'], index=vocab_food)
food_df['graph'] = extend_vocabulary(food_graph_scores, vocab_food)
food_df['combined'] = extend_vocabulary(food_combined_scores, vocab_food)
food_df.tail()

,graph,combined
японский_ADJ,0.0,NaN
японско-минималистический_ADJ,0.0,NaN
яркий_ADJ,1.0,0.0
ярусный_ADJ,NaN,0.0
ясный_ADJ,0.0,NaN


In [47]:
food_df['ok'] = food_df['graph'] == food_df['combined']
food_df[food_df['ok'] == False].dropna()

,graph,combined,ok
глупо_ADJ,0.0,1.0,False
голодный_ADJ,0.0,1.0,False
дурной_ADJ,0.0,1.0,False
замечательный_ADJ,1.0,0.0,False
злой_ADJ,0.0,1.0,False
невкусно_ADJ,0.0,1.0,False
невкусный_ADJ,0.0,1.0,False
незабываемый_ADJ,1.0,0.0,False
необычный_ADJ,1.0,0.0,False
несъедобный_ADJ,0.0,1.0,False


In [48]:
resolve_food = list(food_df[food_df['ok'] == False].dropna().index)
resolve_food

['глупо_ADJ',
 'голодный_ADJ',
 'дурной_ADJ',
 'замечательный_ADJ',
 'злой_ADJ',
 'невкусно_ADJ',
 'невкусный_ADJ',
 'незабываемый_ADJ',
 'необычный_ADJ',
 'несъедобный_ADJ',
 'нормально_ADJ',
 'обидный_ADJ',
 'оригинальный_ADJ',
 'отвратительно_ADJ',
 'плохо_ADJ',
 'плохой_ADJ',
 'удивительный_ADJ',
 'хороший_ADJ',
 'хорошо_ADJ',
 'яркий_ADJ']

In [49]:
ok_food_df = food_df.drop(index=resolve_food).fillna(0)
ok_food_df = pd.DataFrame(ok_food_df['graph'] + ok_food_df['combined'], columns=['correct'])
ok_food_df.tail()

,correct
янтарный_ADJ,0.0
японский_ADJ,0.0
японско-минималистический_ADJ,0.0
ярусный_ADJ,0.0
ясный_ADJ,0.0


In [0]:
resolved_food = [('вкусный_ADJ', 1.0),
                ('глупо_ADJ', 0.0),
                ('голодный_ADJ', 0.0),
                ('дурной_ADJ', 0.0),
                ('замечательный_ADJ', 1.0),
                ('злой_ADJ', 0.0),
                ('невкусно_ADJ', 0.0),
                ('невкусный_ADJ', 0.0),
                ('незабываемый_ADJ', 1.0),
                ('необычный_ADJ', 1.0),
                ('несъедобный_ADJ', 0.0),
                ('нормально_ADJ', 1.0),
                ('обидный_ADJ', 0.0),
                ('оригинальный_ADJ', 1.0),
                ('отвратительно_ADJ', 0.0),
                ('плохо_ADJ', 0.0),
                ('плохой_ADJ', 0.0),
                ('удивительный_ADJ', 1.0),
                ('хороший_ADJ', 1.0),
                ('хорошо_ADJ', 1.0),
                ('яркий_ADJ', 1.0)]

In [56]:
resolved_food_df = pd.DataFrame([el[1] for el in resolved_food], columns=['correct'], index=resolve_food+['вкусный'])
resolved_food_df.tail()

,correct
удивительный_ADJ,0.0
хороший_ADJ,1.0
хорошо_ADJ,1.0
яркий_ADJ,1.0
вкусный,1.0


In [57]:
ok_food_df.append(resolved_food_df)

,correct
авторский_ADJ,0.0
агрессивный_NOUN,1.0
адекватный_ADJ,0.0
адоптировать_ADJ,0.0
ажурный_ADJ,0.0
...,...
удивительный_ADJ,0.0
хороший_ADJ,1.0
хорошо_ADJ,1.0
яркий_ADJ,1.0


In [0]:
ok_food_scores = [tuple(r) for r in ok_food_df.append(resolved_food_df).reset_index().values]

## Combine all

### Service

In [0]:
ok_service_scores += service_n_gram_scores

In [60]:
final_service_scores_df = pd.DataFrame(ok_service_scores, columns=['word', 'score'])
final_service_scores_df['score'] = final_service_scores_df['score'].apply(int)
final_service_scores_df

,word,score
0,авторский_ADJ,0
1,адекватный_ADJ,0
2,адоптировать_ADJ,0
3,азиатский_ADJ,0
4,аккуратненький_ADJ,0
...,...,...
1559,место_NOUN проведение_NOUN для_ADP,0
1560,место_NOUN город_NOUN,0
1561,место_NOUN обслуживание_NOUN с_ADP,0
1562,место_NOUN атмосфера_NOUN,0


In [0]:
final_service_scores_df.to_csv('/content/drive/My Drive/studies/HSE/prog/nlp/proj/final_service_scores.csv', index=False)

### food

In [0]:
ok_food_scores += food_n_gram_scores

In [63]:
final_food_scores_df = pd.DataFrame(ok_food_scores, columns=['word', 'score'])
final_food_scores_df['score'] = final_food_scores_df['score'].apply(int)
final_food_scores_df

,word,score
0,авторский_ADJ,0
1,агрессивный_NOUN,1
2,адекватный_ADJ,0
3,адоптировать_ADJ,0
4,ажурный_ADJ,0
...,...,...
1619,плохо_ADJ не_PART но_CCONJ,0
1620,плохо_ADJ очень_ADV,0
1621,плохо_ADJ не_PART,0
1622,есть_VERB плохо_ADV,0


In [0]:
final_food_scores_df.to_csv('/content/drive/My Drive/studies/HSE/prog/nlp/proj/final_food_scores.csv', index=False)